In [1]:
#include <iostream>
#include <vector>
#include <sstream>

template <class T>
class mapper{
public:
	mapper(){}
protected:	
virtual void run(T&task) = 0;
	virtual void save(T&task, std::ostream&,bool ready) = 0;
	virtual void load(T&task, std::istream&,bool ready) = 0;
public:
	void map() {
		for(auto& task: tasks){
			std::stringstream stream;
			save(task,stream,false);
			load(task,stream,false);

			run(task);
			
			stream.clear();
			save(task,stream,true);
			load(task,stream,true);
		}
	}
	std::vector<T> tasks;
};

In [2]:
struct sqware_task{
	int N;
	int NxN;
}; 

class sqware_mapper : public  mapper<sqware_task>{
public:
	sqware_mapper():mapper(){}
private:
	virtual void  run(sqware_task&task) override { task.NxN = task.N * task.N; }
	virtual void  save(sqware_task&task, std::ostream&out,bool ready) override {
		if(ready) out << task.NxN; else out << task.N;
	}
	virtual void  load(sqware_task&task, std::istream&in,bool ready) override {
		if(ready) in >> task.NxN; else in >> task.N;
	}
};

In [3]:
{
	sqware_mapper mapper_object;

	mapper_object.tasks.resize(10);
	for(int i=0;i<10;i++) mapper_object.tasks[i].N = i+1;

	mapper_object.map();

	for(int i=0;i<10;i++)
		std::cout << mapper_object.tasks[i].N
			<< " ^2 = " 
			<< mapper_object.tasks[i].NxN
			<< std::endl; 
}

1 ^2 = 1
2 ^2 = 4
3 ^2 = 9
4 ^2 = 16
5 ^2 = 25
6 ^2 = 36
7 ^2 = 49
8 ^2 = 64
9 ^2 = 81
10 ^2 = 100
